In [1]:
# pip install beautifulsoup4

# Scrape a list of potions

import requests
from bs4 import BeautifulSoup

def removeCitations(tag):
    for sup in tag.find_all('sup'):
            sup.extract()


def extractList(html):
    tab_elements = html.find_all("div", class_='tabber')

    listItems=[]
    for tab in tab_elements:
        items=tab.css.select("div.wds-tab__content > ul > li")
        for item in items:
            removeCitations(item)
            link = item.find('a')
            if( link ):
                description = item.css.select('ul > li')
                listItems.append({'name':link.text, 'link': link["href"], 'description':  description[0].get_text() if len(description)>0 else '' })

    return listItems

baseUrl = "https://harrypotter.fandom.com"
potionsUrl = "/wiki/List_of_potions"

page = requests.get(baseUrl+potionsUrl)
soup = BeautifulSoup(page.content, "html.parser")

potions=extractList(soup)
print("-- Potions --")
for x in range(len(potions)):
    print(potions[x])



AttributeError: 'NoneType' object has no attribute 'select'

In [ ]:
# grab potion details
for potion in potions:
    print(potion['name'])
    potionPage = requests.get(baseUrl+potion['link'])
    html = BeautifulSoup(potionPage.content, "html.parser")

    potionInfoItems = html.find_all('div', class_='pi-item')

    recipeHeader = html.find(id='Recipe')
    if( recipeHeader ):
        steps=[]
        for valueItem in recipeHeader.find_parent('h2').find_next('ol').find_all('li'):
           steps.append(valueItem.get_text())
        potion['recipe']=steps

    for potionInfo in potionInfoItems:
        removeCitations(potionInfo)
        propertyName=potionInfo['data-source']
        propertyValue=potionInfo.css.select_one('div.pi-data-value')

        valueList = propertyValue.css.select_one('ul')
        if( valueList ):
            values=[]
            for valueItem in valueList.find_all('li'):
                values.append(valueItem.get_text())
            potion[propertyName]=values    
        else:
            potion[propertyName]=propertyValue.get_text()

import json

SAVE_PATH = "data/potions.json"
with open(SAVE_PATH, 'w') as f:
    json.dump(potions, f)
